Run in bash : 
java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -serverProperties StanfordCoreNLP-chinese.properties -preload tokenize,ssplit,pos,lemma,ner,parse -status_port 9001  -port 9001 -timeout 15000

# 匯入packages

In [29]:
import jieba 
from nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser('http://localhost:9001')
from nltk.tree import Tree
import numpy as np

In [110]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
import pandas as pd
import os
from google import google
from collections import Counter
import datetime

ImportError: cannot import name 'google' from 'google' (unknown location)

# 從google上抓出相關連結

In [ ]:
#找出時間的範圍
def get_date_range(date_start,date_end):
    date = []
    start_year = int(date_start.split('/')[0])
    start_month = int(date_start.split('/')[1])
    start_day = int(date_start.split('/')[2])
    end_year = int(date_end.split('/')[0])
    end_month = int(date_end.split('/')[1])
    end_day = int(date_end.split('/')[2])
    date_s = datetime.datetime(start_year,start_month,start_day)
    date_e = datetime.datetime(end_year,end_month,end_day)
    date_range = date_e - date_s
    for i in range(date_range.days+1):
        date.append((date_s+datetime.timedelta(days=i)).strftime("%Y/%m/%d"))
    return date

In [ ]:
#關鍵字串以及時間
def get_google(keyword,date_start,date_end,num_page):
    date = get_date_range(date_start,date_end)
    search_link = {}
    for i in range(len(keyword)):
        link_word = []
        for j in range(len(date)):
            keyword_date = keyword[i]+" "+"\""+date[j]+"\""
            search_results = google.search(keyword_date,num_page)
            for k in search_results:
                link_word.append(k.link)
        search_link[keyword[i]] = link_word
    return search_link

In [ ]:
keyword = ["洗錢","金融詐欺"]
s = "2019/3/15"
e = "2019/3/17"
link = get_google(keyword,s,e,2)

# 從連結抓出相關文章

In [ ]:
def get_html(url):
    try:
        user_agent = 'Mozilla/5.0'
        resp = requests.get(url, headers={'User-Agent': user_agent}, timeout = 30) #回傳為一個request.Response的物件
        resp.endcoding = 'utf8'
        return resp.text
    except:
        return 'ERROR'

In [ ]:
def get_all_text(url):
    try:
        text = get_html(url)
        html = etree.HTML(text)
        no_use_content = html.xpath('//*/text()')
        content = str()
        flag = False #不等於中文字的情況
        count = 0 #想要找出是句中標點的東西，1代表是在句子中
        for i in no_use_content:
            for ch in i:
                if (u'\u4e00' >= ch or ch >= u'\u9fff'):
                    count += 1
                    flag = False
                if (flag == False and count == 1): 
                    content += ch
                if not(u'\u4e00' >= ch or ch >= u'\u9fff'):
                    count = 0
                    flag = True
                    content += ch
        content = delete_ad_rough(content)
        char_list = ['!','?','\＂','＃','＄','％','＆','＊','＜','＝','＞','\'','：','；',' ','\u3000','\xa0','>','<','_']
        for ch in content:
            if ch in char_list:
                content = content.replace(ch,'')
        content = content.replace('\r', '')
        content = content.replace('\t', '')
        content = content.replace('\n', '')
        content = content.replace('\"', '')
        content = content.replace('：', '')
        return content
    except:
        return "ERROR"

In [ ]:
#簡易的清除廣告
def delete_ad_rough(content,limit_num=10):
    content_split = content.split(" ") #先將字串切割
    final_content = str()
    for i in content_split:
        if len(i)>limit_num:           #若字數大於10則判斷不為廣告
            final_content+="。"
            final_content+=i
    return final_content

In [ ]:
#將文章依關鍵字別寫入檔案中
content_list_short = []
w = open("keyword_file.txt", 'w',encoding='UTF-8')
for i in keyword:
    content_list = []
    w.write(i)
    w.write('\n')
    f = open(i + ".txt", "w",encoding='UTF-8')
    for j in link[i]:
        text = get_all_text(j)
        if text!="ERROR":
            content_list.append(text)
    for k in range(len(content_list)):
        a = content_list[k].strip('。')
        f.write(a)
        f.write("\n")
    f.close()
w.close()

# 自檔案讀取文章

In [30]:
def read_text():
    r = open('test_news.txt', 'r')
    text = ''
    for i in r:
        text += i
    sentence = ''
    article = []
    for i in text:   
        if (i != '。') and (i != '\n'):
            sentence += i
        else:
            if sentence != '':
                article.append(sentence)
            sentence = ''
    return article

In [7]:
keyword_list = ["賄賂","貪污"]
article = read_text()

In [8]:
print(article)

['涉收賄數百萬用親友帳戶洗錢\u3000內政部資訊主任與廠商董座遭聲押', '記者劉昌松／台北報導', '內政部資訊中心主任施明德，涉嫌在移民署資訊組長任內，經手電腦網路相關標案時，收受廠商跨越科技賄賂數百萬元，再疑似利用親友戶頭層層轉帳，躲避追查，台北地檢署11日指揮廉政署將施、以及跨越董事長李奇申約談到案，經漏夜訊問，12日上午7點多，以2人各涉犯貪污等罪嫌重大，有勾串共犯證人之虞為由，向法院聲請羈押禁見', '施明德從2009年起，在移民署擔任資訊組組長，2016年升任內政部資訊中心主任，隔年就遭檢舉，在移民署任內經手的至少7項標案涉及不法，北檢2017年才因他涉嫌入出國自動查驗系桶等多項採購弊案，約談後諭令100萬元交保，當時李奇申也是涉案廠商身分，也被諭令100萬元交保', '檢調持續清查他經手案件發現，2009年「外網安全節能設備」，以及2010年的「電腦設備更新」，疑有相同弊端，且勾稽施明德的金流發現，施疑似利用層層轉帳的方式，把收賄贓款漂白，甚至刻意避開直系血親的戶頭，用其他親友戶頭洗錢，規避追查', '台北地檢署11日指揮廉政署大動作搜索21處，約談施、李，以及施的親友等共13人到案說明後，12日分別依涉嫌貪污等罪，向法院聲請將施、李2人羈押禁見', '桃園地檢署偵辦前桃園縣長吳志揚隨行秘書鄒德道等人涉嫌收受楊梅地區的地主100萬元現金及價值200萬元的BMW案，並居中介紹官員協助土地變更案，今偵查終結，依貪污治罪條例起訴將鄒等人起訴', '\xa0桃園市楊梅區仁美里有一片農地，地主們希望能透過《產業創新條例》變更為工業園區開發，地主們計算開發成本後，準備聯合出資4億元活動費，也成立「公庫」各自匯款集結5千萬元', '\xa0其中黃姓地主等人在2012年底開始接觸時任縣長隨行秘書鄒德道（37歲），還致贈一輛價值近2百萬元的BMW及100萬元現金，鄒德道也在該土地變更案中介紹官員讓黃等人熟識並從中協助', '然2014年底，因吳志揚未能連任，鄒德道無從再行使力', '\xa0案經桃園地檢署肅貪專組檢察官吳宗憲指揮法務部廉政署廉政官、桃園市調處追查，再查出楊梅地政事務所葉姓測量助理（50歲）、桃園市桃園地政事務所宋姓測量助理（45歲）也涉嫌收受地主賄款', '\xa0桃園地檢署指出，鄒德道為第16任桃園縣縣長隨行機要秘書，黃姓地主等人、係桃園縣楊梅市

# 使用NER自文章中取出名字

In [ ]:
# %cd /Users/lawrence70726/Desktop/zh-NER-TF-master

In [9]:
# %run main.py

In [10]:
# name_list 

# 找出同時包含名字及關鍵字之句子

In [3]:
def get_sentence(article,keyword_list,name_list):
    sentence_contain_keyword = []
    for i in range(len(article)):
        for j in range(len(keyword_list)):
            if article[i].find(keyword_list[j]) != -1:
                sentence_contain_keyword.append(article[i])            
    sentence_contain_keyword = set(sentence_contain_keyword)
    
    sentence_contain_name = []
    for i in range(len(article)):
        for j in range(len(name_list)):
            if article[i].find(name_list[j]) != -1:
                sentence_contain_name.append(article[i])            
    sentence_contain_name = set(sentence_contain_name)
    
    sentence_wanted = set(sentence_contain_keyword) & set(sentence_contain_name)
    sentence_wanted = list(sentence_wanted)
    return sentence_wanted

In [12]:
keyword_list = ["賄賂","貪污",'洗錢']
name_list = ["劉昌松","施明德","李奇申","鄒德道","吳志揚","吳宗憲"]
sentence_wanted = get_sentence(article, keyword_list, name_list)

#name_list 的格式應改成第n個句子有m個人名(n*m)

In [13]:
print(sentence_wanted)

['桃園地檢署偵辦前桃園縣長吳志揚隨行秘書鄒德道等人涉嫌收受楊梅地區的地主100萬元現金及價值200萬元的BMW案，並居中介紹官員協助土地變更案，今偵查終結，依貪污治罪條例起訴將鄒等人起訴', '內政部資訊中心主任施明德，涉嫌在移民署資訊組長任內，經手電腦網路相關標案時，收受廠商跨越科技賄賂數百萬元，再疑似利用親友戶頭層層轉帳，躲避追查，台北地檢署11日指揮廉政署將施、以及跨越董事長李奇申約談到案，經漏夜訊問，12日上午7點多，以2人各涉犯貪污等罪嫌重大，有勾串共犯證人之虞為由，向法院聲請羈押禁見', '檢調持續清查他經手案件發現，2009年「外網安全節能設備」，以及2010年的「電腦設備更新」，疑有相同弊端，且勾稽施明德的金流發現，施疑似利用層層轉帳的方式，把收賄贓款漂白，甚至刻意避開直系血親的戶頭，用其他親友戶頭洗錢，規避追查']


# 將名字及關鍵字寫入自訂辭典

In [4]:
def write_word(word,freq=10):
    with open('/Users/lawrence70726/Desktop/CS/user_dict.txt','r') as f:
        readword = f.read()
        if word not in readword:
            with open('/Users/lawrence70726/Desktop/CS/user_dict.txt', 'a') as file:
                file.write("\n"+word+" "+str(freq))

In [15]:
for i in name_list:
    write_word(i)
for i in keyword_list:
    write_word(i)

# 匯入自訂辭典

In [2]:
jieba.load_userdict("/Users/lawrence70726/Desktop/CS/user_dict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/yk/b3j7ds1j4fg454ccs84nx0c80000gn/T/jieba.cache
Loading model cost 1.323 seconds.
Prefix dict has been built succesfully.


# 用jieba切字並將句子製成樹狀

In [81]:
def make_tree(sentence,keyword_list):
    token = jieba.cut(sentence, cut_all = False)
    flag = 0
    token_list = list(token)
    for i in keyword_list: 
        if i in token_list:
            flag = 1
    if flag!=1:
        return None
    string = '-'.join(token_list)
    jieba_string = list(parser.parse_text(string))
    string = str(jieba_string[0]).replace('\n','')
    string = string.replace('(PU -)','')
    string = string.split(" ")
    a = str()
    for i in string:
        if i !=" " and i!='':
            a += i
            a += " "
    a = a.replace('(PU -)','')
    tree = Tree.fromstring(a)
    return tree

In [36]:
# For Testing
# test_sentence = u'桃園地檢署偵辦前桃園縣長吳志揚隨行秘書鄒德道等人涉嫌收受楊梅地區的地主100萬元現金及價值200萬元的BMW案，並居中介紹官員協助土地變更案，今偵查終結，依貪污治罪條例起訴將鄒等人起訴'
# test_sentence = u'王俊凱發現顧立雄指控吳志揚協同鄒德道以及李佳恩涉嫌收受楊梅地區的地主100萬元現金及價值200萬元的BMW案，並居中介紹官員協助貪污洗錢'
# test_sentence = u'吳志揚被鄒德道以及王俊凱等人控訴涉嫌洗錢'
# test_sentence = u'吳志揚被王俊凱依貪污治罪條例起訴。'
# test_sentence = u'王俊凱依照貪污治罪條例起訴顧立雄。'
# test_sentence = u'內政部資訊中心主任施明德，涉嫌在移民署資訊組長任內，經手電腦網路相關標案時，收受廠商跨越科技賄賂數百萬元，再疑似利用親友戶頭層層轉帳，躲避追查，台北地檢署11日指揮廉政署將施、以及跨越董事長李奇申約談到案，經漏夜訊問，12日上午7點多，以2人各涉犯貪污等罪嫌重大，有勾串共犯證人之虞為由，向法院聲請羈押禁見'
# test_sentence = u'顧立雄參與反洗錢活動'
# tree = make_tree(test_sentence,keyword_list)

0


In [49]:
print(tree_list)

[Tree('ROOT', [Tree('IP', [Tree('NP', [Tree('NN', ['內政部']), Tree('NP', [Tree('NN', ['資訊']), Tree('NN', ['中心']), Tree('NN', ['主任'])])]), Tree('NP', [Tree('NR', ['施明德'])]), Tree('PU', ['，']), Tree('VP', [Tree('VV', ['涉嫌']), Tree('PP', [Tree('P', ['在']), Tree('NP', [Tree('NN', ['移民']), Tree('NR', ['署資訊']), Tree('NP', [Tree('NR', ['組長'])]), Tree('NR', ['任內']), Tree('PU', ['，']), Tree('PRN', [Tree('IP', [Tree('NP', [Tree('NR', ['經手'])]), Tree('NP', [Tree('NR', ['電腦'])]), Tree('NP', [Tree('NR', ['網路'])]), Tree('NP', [Tree('NR', ['相關'])]), Tree('NP', [Tree('NR', ['標案'])]), Tree('NP', [Tree('NR', ['時'])]), Tree('PU', ['，']), Tree('VP', [Tree('VV', ['收受']), Tree('IP', [Tree('NP', [Tree('NN', ['廠商'])]), Tree('VP', [Tree('VV', ['跨越']), Tree('IP', [Tree('NP', [Tree('NP', [Tree('NN', ['科技'])]), Tree('PRN', [Tree('IP', [Tree('VP', [Tree('VP', [Tree('VV', ['賄賂'])]), Tree('QP', [Tree('CD', ['數百']), Tree('CLP', [Tree('M', ['萬元'])])])])])])]), Tree('PU', ['，']), Tree('ADVP', [Tree('AD', ['再'])]), Tree('

In [ ]:
# tree.pretty_print()
for i in tree_list:
    i.pretty_print()
    print('\n---------------------------------------------------------------------\n')

In [ ]:
# print(tree)
for i in tree_list:
    print(i)
    print('\n---------------------------------------------------------------------\n')

In [47]:
# tree.draw()
for i in tree_list:
    i.draw()

# 判斷

In [6]:
def count_distance(string_tree,from_leaf,to_leaf):
    try:
        tree_pos = string_tree.treepositions('leaves')
        find_node = ()
        for i in tree_pos:
            if string_tree[i]==from_leaf:
                from_leaf_idx = i
            elif string_tree[i]==to_leaf:
                to_leaf_idx = i
        length = len(from_leaf_idx) if len(from_leaf_idx)<len(to_leaf_idx) else len(to_leaf_idx)
        for i in range(length):
            if from_leaf_idx[i]==to_leaf_idx[i]:
                find_node += (from_leaf_idx[i],)
            else:
                break
        length_from_leaf = len(from_leaf_idx) - len(find_node)
        length_to_leaf = len(to_leaf_idx) - len(find_node)
        return length_from_leaf+length_to_leaf
    except:
        return 'Error'

In [7]:
def find_idx(string_tree,name):
    tree_pos = string_tree.treepositions('leaves')
    for i in tree_pos:
        if string_tree[i]==name:
            name_idx = i
    return name_idx

In [8]:
def is_same_label(tree, name_1,name_2):
    name1 = find_idx(tree,name_1)
    name2 = find_idx(tree,name_2)
    find_node_1 = []
    find_node_2 = []
    for i in range(len(name1)):
        if name1[i] == name2[i]:
            find_node_1 += [name1[i]]
            find_node_2 += [name2[i]]
        else:
            find_node_1 += [name1[i]]
            find_node_2 += [name2[i]]
            break
    if (tree[find_node_1].label() == 'NP' and tree[find_node_2].label() == 'NP') or (tree[find_node_1].label() == 'IP' and tree[find_node_2].label() == 'IP'):
        return True
    else:
        return False

In [9]:
def is_leaf(tree,word):
    tree_pos = tree.treepositions('leaves')
    for i in tree_pos:
        if tree[i]==word:
            return True
    return False

In [59]:
def find_bad(tree, name_list, keyword_list):
    try:
        name_bad = {}
        distance_list = []
        closest_name_list = []
        if tree==None:
            return None
        for i in keyword_list:
            if is_leaf(tree,i):
                keyword = i
        if len(name_list) > 1:
            for idx, i in enumerate(name_list):
#                 name_bad[i] = False
                distance_list.append(count_distance(tree,i,keyword))
#                 if distance_list[idx]=='Error':
#                     name_bad[i] = None
#             if all(i=='Error' for i in distance_list):
#                 return closest_name_list
            minidx = np.argmin(distance_list)
            print("minidx= ",minidx)
            for i in range(len(name_list)):
                if i != minidx:
                    dist1 = count_distance(tree,name_list[minidx],keyword)
                    dist2 = count_distance(tree,name_list[i],keyword)
                    if dist2=='Error':
                        continue
                    elif dist1 != dist2:
                        if is_same_label(tree,name_list[minidx],name_list[i]):
                            name_bad[name_list[i]] = True
                    else:
                        name_bad[name_list[i]] = True
                name_bad[name_list[minidx]] = True
            for i in name_bad.items():
                if i[1]==True:
                    closest_name_list.append(i[0])
        else:
            closest_name_list.append(name_list[0])
        return closest_name_list
    except:
        return 'Error'


In [48]:
def judge_active_verb(verb):
    r1 = open('money_laundring_verb.txt', 'r')
    verb_list = []
    for i in r1:
        verb_list.append(i.strip())
    proved_verb = "n"
    if verb in verb_list:
        proved_verb = "y"
    r1.close()
    return proved_verb

In [49]:
def judge_passive_verb(verb):
    r1 = open('money_laundring_passive_verb.txt', 'r')
    verb_list = []
    for i in r1:
        verb_list.append(i.strip())
    proved_verb = "n"
    if verb in verb_list:
        proved_verb = "y"
    r1.close()
    return proved_verb

In [99]:
def judge_sentence(tree,show_name, closest_criminal_list, keyword_list):
    #函數用來判斷句構是否是：名字 +廢字+ 動詞（有洗錢嫌疑的動詞） + 關鍵字/包含關鍵字的子句或廢字
    if len(closest_criminal_list) != 0:
        closest_criminal = closest_criminal_list[-1]
    try:
        leaves_upper_node_indexs = []
        for i in tree.treepositions('leaves'):#treepositions('leaves')的list裡面是tuple
            index = []
            for j in range(len(i)-1):
                index.append(i[j])
            index = tuple(index)
            leaves_upper_node_indexs.append(index)
        words_and_tags = []
        for i in leaves_upper_node_indexs:
            words_and_tags.append(str(tree[i]).strip("(").strip(")").split(" "))
        #print(words_and_tags)#每一個項目長這樣： ['NR', '桃園']
        #找出離名字最近的動詞(這裡可能之後會有問題，但在句中離名字最近會比用樹的距離來判斷好)

        #檢查是否名字在受詞
        passive = False
        closest_criminal_index = 0
        keyword_index = 0
        how_close = 10000
        for i in words_and_tags:
            if i[1] == closest_criminal:
                closest_criminal_index = words_and_tags.index(i)
                break
        for i in words_and_tags:
            for j in keyword_list:
                if i[1] == j and abs(keyword_index - closest_criminal_index) < how_close :
                    keyword_index = words_and_tags.index(i)
                    how_close = abs(keyword_index - closest_criminal_index)
        if keyword_index < closest_criminal_index:
            passive = True


        #若名字在敘事句的主詞位置
        if passive == False:
            name_flag = 0 #假設前面沒有出現過那個名字（一般一個敘事句會提到兩次同個人名字的可能比較小）
            bad_sentence = False
            v = ''
            pass_word = 0
            pass_word_list = []
            for i in words_and_tags:
                if i[1] == closest_criminal:
                    name_flag += 1
                if name_flag == 1:#目的是判斷現在句子位置在要找的人名之後
                    if i[0] == 'VV':
                        v = i[1]
                        if judge_active_verb(v) == 'y':
                            bad_sentence = True
                        break
                    else:
                        pass_word_list.append(i[1])
            pass_word = len(pass_word_list)

        #若名字在敘事句的受詞位置（往回查找）
        if passive == True:
            bad_sentence = False
            v = ''
            pass_word = 0
            pass_word_list = []
            for i in range(closest_criminal_index):
                if words_and_tags[-(len(words_and_tags)-closest_criminal_index)+(-i)][0] == 'VV':
                    v = words_and_tags[-(len(words_and_tags)-closest_criminal_index)+(-i)][1]
                    if judge_passive_verb(v) == "y":
                        bad_sentence = True
                    break
                else:
                    pass_word_list.append(words_and_tags[-(len(words_and_tags)-closest_criminal_index)+(-i)][1])
            pass_word = len(pass_word_list)
        name_and_status = dict()
        for j in show_name:
                name_and_status[j] = False
        for i in closest_criminal_list:
            name_and_status[i] = bad_sentence
    except:
        name_and_status = "ERROR"
    return name_and_status#[bad_sentence, v, pass_word, pass_word_list]

In [46]:
# keyword = "貪污"
# name_list = ["李佳恩","地檢署","吳志揚","鄒德道","王俊凱","顧立雄","施明德","李奇申"]
# name_bad = find_bad(tree,name_list,keyword)
# print(name_bad)
# print(judge_sentence(tree,name_bad))

['王俊凱', '顧立雄']
['Not certain', '', 2, ['。']]


# 計算正確率

In [106]:
def evaluate_accuracy(show_name, name_dict, ans_dict):
    accuracy=0
    for j in range(len(name_dict)):
        flag=0
        for i in show_name[j]:
            if name_dict[j][i]!=ans_dict[j][i]:
                flag=0
                break
            flag=1
        accuracy+=flag
    return accuracy/float(len(name_dict))  

# 資料測試

In [107]:
def organize_data():
    #整理測試資料
    text_data = []
    with open('洗錢測試50句資料csv.csv','r') as r3:
        for i in r3:
            text_data.append(i.strip())
    text_data.remove(text_data[0])
    text_data1 = []
    for i in text_data:
        text_data1.append(i.split(","))

    for i in range(len(text_data1)):
        text_data1[i][1] = text_data1[i][1].split("，")
        text_data1[i][0] = text_data1[i][0].split("，")
    sentence_wanted = [text_data1[i][2] for i in range(len(text_data1))]
    show_name = []
    for i in range(len(text_data1)):
        show_name.append(text_data1[i][1])
    ans_dict = []
    for i in range(len(text_data1)):
        answer_dict = dict()
        for j in text_data1[i][0]:
            if j in answer_dict:
                no_use = 0
            else:
                answer_dict[j] = True

        for k in text_data1[i][1]:
            if k in answer_dict:
                no_use = 0
            else:
                answer_dict[k] = False
        #del answer_dict["無"] 
        if "無" in answer_dict:
            del answer_dict["無"]
        ans_dict.append(answer_dict)
   
    return sentence_wanted, show_name, ans_dict

# Main

In [108]:
def demo():
    try:
        keyword_list = ["賄賂","貪污", "洗錢", "詐欺", "境外轉帳", "詐騙"]
        sentence_wanted, show_name, ans_dict = organize_data()
        name_dict = []
        #加入結巴詞庫
        for i in show_name:
            for j in i:
                write_word(j)

        for i in keyword_list:
            write_word(i)
        #讀取結巴
        jieba.load_userdict("/Users/lawrence70726/Desktop/CS/user_dict.txt")
        for i,sentence in enumerate(sentence_wanted):
            parser = CoreNLPParser('http://localhost:9001')
            name = {}
            tree = make_tree(sentence,keyword_list)
            if tree==None:
                closest_name_list = find_bad(tree, show_name[i], keyword_list)
            if closest_name_list == None:
                for j in show_name[i]:
                    name[j] = False
            else:
                name = judge_sentence(tree, show_name[i],closest_name_list, keyword_list)
            name_dict.append(name)
        accuracy = evaluate_accuracy(show_name,name_dict,ans_dict)
        print("Accuracy= ",accuracy)   
        return name_dict
    except:
        print("Error :(((")

In [109]:
#main
name_dict = demo()

Accuracy=  0.5934959349593496


# 待解決事項：
* 被動式要進行修正（例如：test_sentence = u'吳志揚被鄒德道以及王俊凱等人控訴涉嫌洗錢'）
* 若距離皆一樣，但意思不一樣（例如：test_sentence = u'吳志揚被鄒德道以及王俊凱等人控訴涉嫌洗錢'）
* 子句中有負面的詞語
* 取得句子時即標記其keyword_list和name_list以節省後面運算的時間
* NER模組擺進去
* 擴充反洗錢條例相關字典